In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import random
import numpy as np
import pandas as pd

In [2]:
import data.data_retriever as data_retriever

In [3]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    
set_seed(0)

In [4]:
df = data_retriever.get_data('USDC')
df['unix_time'] = pd.to_datetime(df['date-time']).astype(int) / 10**9

cols_to_keep = [
    'unix_time',
    'liquidityRate_avg',
    'variableBorrowRate_avg',
    'utilizationRate_avg',
    'close_price',
    'volume',
]

crypto_df = df[cols_to_keep]
crypto_df.head()

,unix_time,liquidityRate_avg,variableBorrowRate_avg,utilizationRate_avg,close_price,volume
0,1.678752e+09,0.005627,0.014052,0.444227,0.9990,1.218521e+06
1,1.678756e+09,0.008435,0.020909,0.447503,0.9991,1.459872e+06
2,1.678759e+09,0.009119,0.022602,0.447697,0.9992,2.188143e+06
3,1.678763e+09,0.007338,0.017976,0.453755,0.9993,2.800380e+06
4,1.678766e+09,0.006334,0.014046,0.464677,0.9994,1.048281e+06


# Dataloader

In [5]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data: pd.DataFrame, window_size: int, target_columns, forecast_size: int = 1):
        self.window_size = window_size
        self.forecast_size = forecast_size
        self.target_columns = target_columns  
        
        data_array = data.values
    
        target_indices = [data.columns.get_loc(col) for col in target_columns]  

        self.X, self.y = [], []
        for i in range(len(data_array) - window_size - forecast_size + 1):
            self.X.append(data_array[i : i + window_size])
            
            if len(target_indices) == 1:
                self.y.append(data_array[i + window_size : i + window_size + forecast_size, target_indices[0]].reshape(-1, 1))
            else:
                self.y.append(data_array[i + window_size : i + window_size + forecast_size, target_indices])

        self.X = np.array(self.X)
        self.y = np.array(self.y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (torch.tensor(self.X[idx], dtype=torch.float32), 
                torch.tensor(self.y[idx], dtype=torch.float32))

In [6]:
def create_dataloaders(
        data, 
        target_columns,
        window_size=168,
        forecast_size=1,
        batch_size=32, 
        val_size=0.1,
        test_size=0.1):
    total_size = len(data)
    train_size = int((1 - val_size - test_size) * total_size)
    val_size = int(val_size * total_size)
    
    train_data = data[:train_size]
    val_data = data[train_size:train_size + val_size]
    test_data = data[train_size + val_size:]
    
    train_dataset = TimeSeriesDataset(train_data, window_size, target_columns, forecast_size)
    val_dataset = TimeSeriesDataset(val_data, window_size, target_columns, forecast_size)
    test_dataset = TimeSeriesDataset(test_data, window_size, target_columns, forecast_size)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, val_loader, test_loader

# Model

In [7]:
class LSTMForecaster(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size=2, forecast_size=1, dropout=0.2):
        super(LSTMForecaster, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.forecast_size = forecast_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :] 
        
        forecast = []
        for _ in range(self.forecast_size):
            out = self.fc(out)
            forecast.append(out.unsqueeze(1))
        
        forecast = torch.cat(forecast, dim=1) 
        return forecast

# Trainer

In [8]:
def train_model(model, train_loader, val_loader, num_epochs=20, learning_rate=0.001, device="cuda", patience=10, lr_patience=5, lr_decay_factor=0.1):
    model.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=lr_decay_factor, patience=lr_patience, verbose=True)

    best_val_loss = float("inf")
    epochs_without_improvement_es = 0
    epochs_without_improvement_lr = 0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            y_pred = model(X_batch)

            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss /= len(train_loader)

        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for X_val, y_val in val_loader:
                X_val, y_val = X_val.to(device), y_val.to(device)

                y_pred = model(X_val)
                loss = criterion(y_pred, y_val)
                val_loss += loss.item()

        val_loss /= len(val_loader)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.8f}, Val Loss: {val_loss:.8f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_without_improvement_es = 0
            epochs_without_improvement_lr = 0
        else:
            epochs_without_improvement_es += 1
            epochs_without_improvement_lr += 1

        if epochs_without_improvement_es >= patience:
            print(f"Early stopping at epoch {epoch+1} due to no improvement in validation loss for {patience} epochs.")
            break

        if epochs_without_improvement_lr >= lr_patience:
            print(f"Validation loss plateaued for {lr_patience} epochs. Reducing learning rate by factor of {lr_decay_factor}.")
            scheduler.step(val_loss)
            epochs_without_improvement_lr = 0

# Evaluator

In [9]:
def evaluate_model(model, test_loader, device="cuda"):
    model.to(device)
    model.eval()

    criterion = nn.MSELoss(reduction='none')
    total_samples = 0
    total_mape = torch.zeros(model.output_size, device=device) 
    total_mse = torch.zeros(model.output_size, device=device) 

    with torch.no_grad():
        for X_test, y_test in test_loader:
            X_test, y_test = X_test.to(device), y_test.to(device)

            y_pred = model(X_test)

            mse_per_element = criterion(y_pred, y_test)
            mape_per_output = torch.mean(torch.abs((y_test - y_pred) / (y_test + 1e-8)), dim=(0, 1)) * 100

            total_mape += mape_per_output * X_test.size(0)
            total_samples += X_test.size(0)

            mse_per_output = mse_per_element.mean(dim=(0, 1)) 
            total_mse += mse_per_output * X_test.size(0)

    average_mse = total_mse.cpu().numpy() / total_samples
    average_mape = total_mape.cpu().numpy() / total_samples

    print(f"Test MSE (for each output dimension): {average_mse}")
    print(f"Test MAPE (for each output dimension): {average_mape}")

    return average_mse, average_mape


# Training

In [10]:
def train_and_evaluate(
        crypto_df,
        input_size=6, 
        hidden_size=32, 
        num_layers=4, 
        output_size=2, 
        forecast_size=1, 
        dropout=0.2,
        target_columns=['liquidityRate_avg', 'variableBorrowRate_avg'],
        window_size=168,
        batch_size=32,
        val_size=0.1,
        test_size=0.1,
        num_epochs=100,
        learning_rate=0.001,
        device="cuda" if torch.cuda.is_available() else "cpu",
        patience=10,
        lr_patience=5,
        lr_decay_factor=0.1,
):

    train_loader, val_loader, test_loader = create_dataloaders(
        crypto_df,
        target_columns=target_columns,
        window_size=window_size,
        forecast_size=forecast_size,
        batch_size=batch_size,
        val_size=val_size,
        test_size=test_size
    )
    
    model = LSTMForecaster(
        input_size, 
        hidden_size, 
        num_layers, 
        output_size, 
        forecast_size, 
        dropout
    ).to(device)
    
    train_model(
        model, 
        train_loader, 
        val_loader, 
        num_epochs=num_epochs, 
        learning_rate=learning_rate, 
        device=device, 
        patience=patience, 
        lr_patience=lr_patience, 
        lr_decay_factor=lr_decay_factor
    )

    evaluate_model(model, test_loader, device=device)

In [11]:
# window size of 1 day
train_and_evaluate(crypto_df, window_size=24)

C:\Users\siam\anaconda3\envs\aave\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/100, Train Loss: 0.00133152, Val Loss: 0.00271658
Epoch 2/100, Train Loss: 0.00128902, Val Loss: 0.00286419
Epoch 3/100, Train Loss: 0.00127949, Val Loss: 0.00294469
Epoch 4/100, Train Loss: 0.00128006, Val Loss: 0.00272830
Epoch 5/100, Train Loss: 0.00127436, Val Loss: 0.00259581
Epoch 6/100, Train Loss: 0.00128120, Val Loss: 0.00289617
Epoch 7/100, Train Loss: 0.00127758, Val Loss: 0.00312897
Epoch 8/100, Train Loss: 0.00127447, Val Loss: 0.00286827
Epoch 9/100, Train Loss: 0.00127553, Val Loss: 0.00271369
Epoch 10/100, Train Loss: 0.00127802, Val Loss: 0.00275847
Validation loss plateaued for 5 epochs. Reducing learning rate by factor of 0.1.
Epoch 11/100, Train Loss: 0.00127334, Val Loss: 0.00282537
Epoch 12/100, Train Loss: 0.00127617, Val Loss: 0.00288639
Epoch 13/100, Train Loss: 0.00127133, Val Loss: 0.00286591
Epoch 14/100, Train Loss: 0.00127263, Val Loss: 0.00289460
Epoch 15/100, Train Loss: 0.00127172, Val Loss: 0.00289496
Early stopping at epoch 15 due to no improv

In [12]:
# window size of 1 week
train_and_evaluate(crypto_df, window_size=7*24)

C:\Users\siam\anaconda3\envs\aave\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/100, Train Loss: 0.00138886, Val Loss: 0.00333181
Epoch 2/100, Train Loss: 0.00128356, Val Loss: 0.00316129
Epoch 3/100, Train Loss: 0.00127711, Val Loss: 0.00290112
Epoch 4/100, Train Loss: 0.00128248, Val Loss: 0.00300550
Epoch 5/100, Train Loss: 0.00127263, Val Loss: 0.00303386
Epoch 6/100, Train Loss: 0.00128477, Val Loss: 0.00302591
Epoch 7/100, Train Loss: 0.00127840, Val Loss: 0.00280287
Epoch 8/100, Train Loss: 0.00127131, Val Loss: 0.00302498
Epoch 9/100, Train Loss: 0.00127337, Val Loss: 0.00288837
Epoch 10/100, Train Loss: 0.00127188, Val Loss: 0.00293777
Epoch 11/100, Train Loss: 0.00127051, Val Loss: 0.00323483
Epoch 12/100, Train Loss: 0.00127262, Val Loss: 0.00321931
Validation loss plateaued for 5 epochs. Reducing learning rate by factor of 0.1.
Epoch 13/100, Train Loss: 0.00127583, Val Loss: 0.00300165
Epoch 14/100, Train Loss: 0.00127103, Val Loss: 0.00321328
Epoch 15/100, Train Loss: 0.00127460, Val Loss: 0.00304872
Epoch 16/100, Train Loss: 0.00126791, Val L

In [13]:
# window size of 1 month
train_and_evaluate(crypto_df, window_size=30*24)

C:\Users\siam\anaconda3\envs\aave\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/100, Train Loss: 0.00147058, Val Loss: 0.00359548
Epoch 2/100, Train Loss: 0.00128096, Val Loss: 0.00408505
Epoch 3/100, Train Loss: 0.00127770, Val Loss: 0.00388415
Epoch 4/100, Train Loss: 0.00127662, Val Loss: 0.00405944
Epoch 5/100, Train Loss: 0.00127861, Val Loss: 0.00466426
Epoch 6/100, Train Loss: 0.00127905, Val Loss: 0.00397106
Validation loss plateaued for 5 epochs. Reducing learning rate by factor of 0.1.
Epoch 7/100, Train Loss: 0.00127543, Val Loss: 0.00421714
Epoch 8/100, Train Loss: 0.00127287, Val Loss: 0.00357628
Epoch 9/100, Train Loss: 0.00127536, Val Loss: 0.00433844
Epoch 10/100, Train Loss: 0.00127424, Val Loss: 0.00428045
Epoch 11/100, Train Loss: 0.00127378, Val Loss: 0.00422194
Epoch 12/100, Train Loss: 0.00127290, Val Loss: 0.00406339
Epoch 13/100, Train Loss: 0.00127487, Val Loss: 0.00381324
Validation loss plateaued for 5 epochs. Reducing learning rate by factor of 0.1.
Epoch 14/100, Train Loss: 0.00127367, Val Loss: 0.00391980
Epoch 15/100, Train L